In [1]:
#importing all the necessary libraraies
import numpy as np
import tensorflow as tf
from tensorflow import keras
print("Tensorflow version -->" +str(tf.__version__))
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
%matplotlib inline
import math

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow version -->1.12.0


In [2]:
#Loading the dataset and preproceesing it
fashion_mnist = keras.datasets.fashion_mnist
(x_train_orig, Y_train), (x_test_orig, Y_test) = fashion_mnist.load_data()
#Now standardizing and sclaing the images
print("Printing the original shapes")
print("For trainng X {} and Y is  {}".format(x_train_orig.shape,Y_train.shape))
print("For testing X {} and Y is {}".format(x_test_orig.shape,Y_test.shape))

Printing the original shapes
For trainng X (60000, 28, 28) and Y is  (60000,)
For testing X (10000, 28, 28) and Y is (10000,)


In [3]:
#Resizing and one hot encoding the images
X_train=x_train_orig.reshape(x_train_orig.shape[0],x_train_orig.shape[1]*x_train_orig.shape[2])
X_test=x_test_orig.reshape(x_test_orig.shape[0],x_test_orig.shape[1]*x_test_orig.shape[2])
X_train=X_train/255.0
X_test=X_test/255.0
lb=LabelBinarizer()
Y_train=lb.fit_transform(Y_train)
Y_test=lb.transform(Y_test)
print("The X_train {} and X_test {}".format(X_train.shape,X_test.shape))
print("One hot encoding results in Y_train {} and Y_test {}".format(Y_train.shape,Y_test.shape))

The X_train (60000, 784) and X_test (10000, 784)
One hot encoding results in Y_train (60000, 10) and Y_test (10000, 10)


In [4]:
#Shuffling and Randomizing batches
def random_mini_batches(X, Y,seed, mini_batch_size = 64):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    #The Code is Transformed for m,n_x Case#
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:]
    shuffled_Y = Y[permutation,:].reshape((m,Y.shape[1]))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [5]:
#Designing The placeholders now
def placeholder(n_x,n_y,keep_prob):
    X=tf.placeholder(tf.float32,[None,n_x],name="X")
    Y=tf.placeholder(tf.float32,[None,n_y],name="Y")
    keep_prob=tf.placeholder(tf.float32,name="keep_prob")
    return X,Y,keep_prob

In [6]:
tf.reset_default_graph()
X,Y,keep_prob=placeholder(784,10,1)
print("X = {}"+str(X))
print("Y = {}"+str(Y))
print("Keep Probability = {}"+str(keep_prob))

X = {}Tensor("X:0", shape=(?, 784), dtype=float32)
Y = {}Tensor("Y:0", shape=(?, 10), dtype=float32)
Keep Probability = {}Tensor("keep_prob:0", dtype=float32)


In [7]:
#Initializing the weights
def initialize_weights():
    W1=tf.get_variable("W1",[784,128],initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b1=tf.get_variable("b1",[128],initializer=tf.zeros_initializer())
    W2=tf.get_variable("W2",[128,256],initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b2=tf.get_variable("b2",[256],initializer=tf.zeros_initializer())
    W3=tf.get_variable("W3",[256,128],initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b3=tf.get_variable("b3",[128],initializer=tf.zeros_initializer())
    W4=tf.get_variable("W4",[128,10],initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b4=tf.get_variable("b4",[10],initializer=tf.zeros_initializer())
    
    parameters={"W1":W1,"b1":b1,"W2":W2,"b2":b2,"W3":W3,"b3":b3,"W4":W4,"b4":b4}
    
    return parameters

In [8]:
tf.reset_default_graph()
X,Y,keep_prob=placeholder(784,10,1)
parameters=initialize_weights()
print("W1 = {}" +str(parameters["W1"]))
print("b1 = {}" +str(parameters["b1"]))
print("------------------------------")
print("b4 = {}" +str(parameters["W4"]))
print("b4 = {}" +str(parameters["b4"]))

W1 = {}<tf.Variable 'W1:0' shape=(784, 128) dtype=float32_ref>
b1 = {}<tf.Variable 'b1:0' shape=(128,) dtype=float32_ref>
------------------------------
b4 = {}<tf.Variable 'W4:0' shape=(128, 10) dtype=float32_ref>
b4 = {}<tf.Variable 'b4:0' shape=(10,) dtype=float32_ref>


In [9]:
#Now Doing the feed forward 
def feed_forward(X,parameters,keep_prob):
    Z1=tf.add(tf.matmul(X,parameters["W1"]),parameters["b1"])
    A1=tf.nn.relu(Z1)
    A1d=tf.nn.dropout(A1,keep_prob)
    Z2=tf.add(tf.matmul(A1,parameters["W2"]),parameters["b2"])
    A2=tf.nn.relu(Z2)
    A2d=tf.nn.dropout(A2,keep_prob)
    Z3=tf.add(tf.matmul(A2,parameters["W3"]),parameters["b3"])
    A3=tf.nn.relu(Z3)
    A3d=tf.nn.dropout(A3,keep_prob)
    Z4=tf.add(tf.matmul(A3,parameters["W4"]),parameters["b4"])
    
    return Z4

In [10]:
tf.reset_default_graph()
X,Y,keep_prob=placeholder(784,10,1)
parameters=initialize_weights()
Z4=feed_forward(X,parameters,1)
print("Z4 is {}".format(Z4))

Z4 is Tensor("Add_3:0", shape=(?, 10), dtype=float32)


In [11]:
def compute_cost(Z4,Y):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z4,labels=Y))
    return cost

In [12]:
tf.reset_default_graph()
X,Y,keep_prob=placeholder(784,10,1)
parameters=initialize_weights()
Z4=feed_forward(X,parameters,1)
cost=compute_cost(Z4,Y)
print("Cost is {}".format(cost))

Cost is Tensor("Mean:0", shape=(), dtype=float32)


In [19]:
def neural_network_model(X_train,Y_train,X_test,Y_test,number_of_epoch,mini_batch_size,learning_rate,dropout_prob,print_cost=True):
    (m,n_x)=X_train.shape
    (n_y)=Y_train.shape[1]
    global_step=tf.Variable(0,trainable=False)
    costs=[]
    seed=3
    np.random.seed(1)
    tf.random.set_random_seed(1)
    
    #First using the placeholders
    X,Y,keep_prob=placeholder(n_x,n_y,dropout_prob)
    
    #Next initializing the weights
    parameters=initialize_weights()
    
    #Next doing the first forward propagation
    Z4=feed_forward(X,parameters,keep_prob)
    
    #Next Computing the cost
    cost=compute_cost(Z4,Y)
    
    #The learning rate decay
    lr=tf.train.exponential_decay(learning_rate,decay_rate=0.96,decay_steps=2000,global_step=global_step,staircase=True)
    
    #Which optimizer we will use
    optimizer=tf.train.AdamOptimizer(beta1=0.98,beta2=0.999,epsilon=1e-08,learning_rate=lr,name="Adam").minimize(cost,global_step)\
    
    #the initializer which we will use
    init=tf.global_variables_initializer()
    
    #Starting the session
    with tf.Session() as sess:
        sess.run(init)
        print("-x-x-x-x-x-x-x-x-x-x-x-x-x-INITIALIZED-x-x-x-x-x-x-x-x-x-x-x-x-x-x--x-x-x-x-x")
        num_of_mini_batches=math.floor(m/mini_batch_size)
        print("The model will be trained for {} epochs and the number of mini batches are {}".format(number_of_epoch,num_of_mini_batches))
        for epoch in range(number_of_epoch):
            epoch_cost=0
            minibatches=random_mini_batches(X_train,Y_train,seed,mini_batch_size)
            seed+=1
            for minibatch in minibatches:
                (X_minibatch,Y_minibatch)=minibatch
                _,minibatchcost=sess.run([optimizer,cost],feed_dict={X:X_minibatch,Y:Y_minibatch,keep_prob:dropout_prob})
                epoch_cost+=minibatchcost/num_of_mini_batches
            #Adding the cost
            if print_cost==True and epoch%100==0:
                print("For the epoch {} the cost is {} with dropout probability of {}".format(epoch,epoch_cost,dropout_prob))
            if print_cost==True and epoch%5==0:
                costs.append(epoch_cost)
        print("-x-x-x-x-x-x-x-x-x-x-x-x-x MODEL TRAINED-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x")
        #printing the cost vs iteration curve
        
        print("the cost V/s Iterationn Curve Is -->")
        plt.plot(np.squeeze(costs))
        plt.xlabel("Iterations in 10's")
        plt.ylabel("Cost")
        plt.title("Cost for the learning rate {}".format(learning_rate))
        plt.show()
        
        #Saving the parameters
        print("Saving the Parameters!")
        parameters=sess.run(parameters)
        
        #Now finding the predictions and accuracy
        print("Calculating the accuracy--->")
        correct_predictions=tf.equal(tf.argmax(Z4,1),tf.argmax(Y,1))
        accuracy=tf.reduce_mean(tf.cast(correct_predictions,"float"))
        print("The training accuracy is {}".format(accuracy.eval({X:X_train,Y:Y_train,keep_prob:dropout_prob})))
        print("The testing accuracy is {}".format(accuracy.eval({X:X_test,Y:Y_test,keep_prob:1.0})))
        
        return parameters

In [ ]:
tf.reset_default_graph()
parameters=neural_network_model(X_train,Y_train,X_test,Y_test,dropout_prob=1.0,learning_rate=0.1,mini_batch_size=64,number_of_epoch=500,print_cost=True)

-x-x-x-x-x-x-x-x-x-x-x-x-x-INITIALIZED-x-x-x-x-x-x-x-x-x-x-x-x-x-x--x-x-x-x-x
The model will be trained for 500 epochs and the number of mini batches are 937
For the epoch 0 the cost is 3.749418838301485 with dropout probability of 1.0
For the epoch 100 the cost is 2.306520904013799 with dropout probability of 1.0
For the epoch 200 the cost is 2.3053317337178623 with dropout probability of 1.0
For the epoch 300 the cost is 2.3050917054952933 with dropout probability of 1.0
For the epoch 400 the cost is 2.305050681978302 with dropout probability of 1.0
